In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%matplotlib inline
import numpy as np
import pandas as pd 
from glob import glob 
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
from skimage.io import imread
from keras.preprocessing.image import ImageDataGenerator

dataDir = '/content/drive/My Drive/GitHub Repositories/Transfer Learning and Fine Tuning with Pre-Trained Networks/datasets/x-rays'
trainDir = os.path.join(dataDir, 'train')
validDir = os.path.join(dataDir, 'test')

IMG_SIZE = 224 # slightly smaller than vgg16 normally expects
datagen = ImageDataGenerator(rescale=1./255, 
                             samplewise_center=False, 
                             samplewise_std_normalization=False, 
                             horizontal_flip = True, 
                             vertical_flip = False, 
                             height_shift_range = 0.15, 
                             width_shift_range = 0.15, 
                             rotation_range = 5, 
                             shear_range = 0.01,
                             fill_mode = 'nearest',
                             zoom_range=0.1,  
                             )

train_gen = datagen.flow_from_directory(
    directory=trainDir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=2020)

valid_gen = datagen.flow_from_directory(
    directory=validDir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=2020)




Found 721 images belonging to 2 classes.
Found 79 images belonging to 2 classes.


In [0]:
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda
from keras.layers import BatchNormalization
from keras.models import Model

img_shape = (IMG_SIZE, IMG_SIZE, 3)
inputs = Input(img_shape)
num_classes = 2

base_pretrained_model = VGG16(input_shape =  img_shape, include_top = False, weights = 'imagenet')
base_pretrained_model.trainable = False
outputs = base_pretrained_model(inputs)
outputs = BatchNormalization()(outputs)
outputs = GlobalAveragePooling2D()(outputs)
outputs = Dense(num_classes, activation = 'softmax')(outputs)
model = Model(inputs = [inputs], outputs = [outputs])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()





58892288/58889256 [==============================] - 6s 0us/step










Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 512)         2048      
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,717,762
Trainable params: 2,050
Non-trainable params: 14,715,712
_________________________________________________________________


In [0]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('/content/drive/My Drive/GitHub Repositories/Visual Explaination from Deep Neural Networks/CAM Approach/saved models/CAM-{epoch:02d}-{val_acc:.4f}.h5',\
                             monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size

model.fit_generator(train_gen, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_gen, validation_steps=STEP_SIZE_VALID,
                    epochs = 15, callbacks = [checkpoint])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 5 could not be retrieved. It could be because a worker has died.
  UserWarning)


 1/22 [>.............................] - ETA: 33:21 - loss: 0.6855 - acc: 0.5000

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 15 could not be retrieved. It could be because a worker has died.
  UserWarning)


 2/22 [=>............................] - ETA: 22:28 - loss: 0.6886 - acc: 0.5469

...

21/22 [===========================>..] - ETA: 1:04 - loss: 0.6210 - acc: 0.6634

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 10 could not be retrieved. It could be because a worker has died.
  UserWarning)


22/22 [==============================] - 1399s 64s/step - loss: 0.6141 - acc: 0.6689 - val_loss: 0.6378 - val_acc: 0.6250

Epoch 00001: val_acc improved from -inf to 0.62500, saving model to /content/drive/My Drive/GitHub Repositories/Visual Explaination from Deep Neural Networks/CAM Approach/saved models/CAM-01-0.6250.h5
Epoch 2/15


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 0 could not be retrieved. It could be because a worker has died.
  UserWarning)


 1/22 [>.............................] - ETA: 19:33 - loss: 0.4840 - acc: 0.7812